# Load and Clean the Data

In [1]:
%run ./preprocessing.ipynb

E    52268
K    13372
Name: customer_gender, dtype: int64


In [2]:
df_male.shape, df_female.shape

((52268, 42), (13372, 42))

# Self-Supervised Similarity Measure

In [3]:
%run ./similarity.ipynb

E    52268
K    13372
Name: customer_gender, dtype: int64


2022-05-09 19:18:53.853022: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-09 19:18:53.853079: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-09 19:18:53.877816: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "autoencoder_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  1388      
                                                                 
 decoder (Decoder)           multiple                  439       
                                                                 
Total params: 1,827
Trainable params: 1,827
Non-trainable params: 0
_________________________________________________________________
Model: "autoencoder_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  1388      
                                                                 
 decoder (Decoder)           multiple                  439       
                                                                 
Total params: 1,827
Trainabl

# Matching Algorithm

In [4]:
%run ./matching_algorithm.ipynb

cannot import name 'linear_sum_assignment' from 'scipy.sparse.csgraph' (/home/mmaddox/.local/lib/python3.8/site-packages/scipy/sparse/csgraph/__init__.py)
name 'linear_sum_assignment' is not defined


In [7]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    """ Create a visualization of a progress bar updates according to completion status
    :param it: job you are trying to create a progress bar for
    :type obj (sequence or collection)
    :param prefix: The text to display to the left of the status bar
    :type str
    :param size: total length of the progress bar
    :type int
    :param file: what to display/write the progress bar to
    :type output stream
    :return: job you are trying to create a progress bar for
    :rtype: obj (sequence or collection)
    """
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [ ]:
from pulp import LpVariable, LpProblem, LpMinimize, value, lpSum, LpBinary, LpStatus

def hungarian_method(matrix_a, matrix_b):
    print('Calculating distance')
    dist = cdist(matrix_a, matrix_b, 'euclidean') # (matrix_a.shape[0], matrix_b.shape[0])
    
    print('Starting algorithm')
    prob = LpProblem("Matching", LpMinimize)
    
    variable_names = [str(i)+','+str(j) for i in range(matrix_a.shape[0]) for j in range(matrix_b.shape[0])]
    variable_names.sort()
    
    relations = LpVariable.matrix("X", variable_names, cat = "Binary")
    allocation = np.array(relations).reshape(matrix_a.shape[0],matrix_b.shape[0])
    
    prob += lpSum(allocation*dist)
    for i in progressbar(range(matrix_a.shape[0])):
        prob += lpSum([allocation[i,j] for j in range(matrix_b.shape[0])]) == 1
        prob += lpSum([allocation[j,i] for j in range(matrix_a.shape[0])]) == 1
        prob += lpSum(allocation[i,j] for j in range(matrix_b.shape[0])) >= 0
    
    
    prob.writeLP("matching_prob.lp")
    prob.solve()
    
    print( 'Status:\t', LpStatus[prob.status] )
    print( 'Objective Value:\t', prob.objective.value() )
    
    print(matrix_a.index)
    
    matrix_b_indices = np.array([matrix_b.index[int(v.name.split(',')[-1])] for v in prob.variables() if bool(v.value())])
    pd.DataFrame(np.dstack([matrix_a.index.values, matrix_b_indices]).reshape(matrix_a.shape[0], -1)).to_csv(os.path.join(data_path, 'matches-sample.csv'), index = False)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mmaddox/.local/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/a66b3c4e4d6b410293972b2ab2615cfe-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/a66b3c4e4d6b410293972b2ab2615cfe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3005 COLUMNS
At line 6003006 RHS
At line 6006007 BOUNDS
At line 7006008 ENDATA
Problem MODEL has 3000 rows, 1000000 columns and 3000000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 15144.8 - 21.79 seconds
Cgl0004I processed model has 2000 rows, 1000000 columns (1000000 integer (1000000 of which binary)) and 2000000 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 15144.8
Cbc0038I Before mini branch and bound, 1000000 integers at bound fixed and 0 continuous
Cbc0038I Mini bran

In [113]:
encoded_sample_males = encoded_males.sample(1000)
encoded_sample_females = encoded_females.sample(1000)

hungarian_method(encoded_sample_males, encoded_sample_females)

Calculating distance
Starting algorithm
[############################################################] 1000/1000



KeyboardInterrupt

